In [67]:
import os
import sys
import json

from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, concat_ws, when, udf
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, ArrayType
import pyspark.sql.functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Create SparkSession

In [6]:
spark = SparkSession.builder.appName('Recommendation_system').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read Data From S3

In [22]:
ratingSchema = StructType([StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", FloatType(), True), 
    StructField("timestamp", IntegerType(), True)])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
ratings = spark.read.csv("s3://ntuee-cloudcompute-movie-recommendation/ratings.csv", header=True, schema=ratingSchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Build Model and Training

In [18]:
als = ALS(maxIter=5, userCol="userId", itemCol="movieId", ratingCol="rating")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
model = als.fit(ratings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Predict and Save

In [36]:
recommendedMovies = model.recommendForAllUsers(10)
recommendedUsers = model.recommendForAllItems(10)
userFactors = model.userFactors
itemFactors = model.itemFactors

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
recommendedMovies.repartition(1).write.json("s3://ntuee-cloudcompute-movie-recommendation/recommendedMovies")
recommendedUsers.repartition(1).write.json("s3://ntuee-cloudcompute-movie-recommendation/recommendedUsers")
userFactors.repartition(1).write.json("s3://ntuee-cloudcompute-movie-recommendation/userFactors")
itemFactors.repartition(1).write.json("s3://ntuee-cloudcompute-movie-recommendation/itemFactors")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…